In [165]:
import numpy as np
import cv2
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
def get_img_contour_thresh(img):
    x, y, w, h = 0, 0, 300, 300
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh1 = thresh1[y:y + h, x:x + w]
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return img, contours, thresh1

In [23]:
! ls 

Digit-Recognizer             project_notes
README.md                    requirements.txt
data                         update_one.docx
dig_rec_scratch.ipynb        update_one.pdf
digit-recognizer-copy        update_one.txt
final-report-dl-2018.pdf     webcam_capture.py
logistic_regression_torch.py


In [24]:
import logistic_regression_torch as lr_torch

SyntaxError: invalid syntax (logistic_regression_torch.py, line 48)

In [8]:
path = Path('data/')

In [9]:
??MNIST

In [10]:
mnist = MNIST(path, download=True)

Processing...
Done!


In [13]:
mnist

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): None
    Target Transforms (if any): None

In [14]:
type(mnist)

torchvision.datasets.mnist.MNIST

In [142]:
batch_size = 64
test_batch_size = 64 # why are these different?

In [143]:
train_dl = torch.utils.data.DataLoader(
    datasets.MNIST(path, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(
    datasets.MNIST(path, train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

In [219]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__() # why are we supering ourselves?
        self.linear = nn.Linear(input_size, num_classes)


    def forward(self, x):
        # not sure it matters which one we use
        out = F.log_softmax(self.linear(x), dim = 1) # is this what I want? or wrap it in F.log_softmax()? 
        #out = self.linear(x)
        return out

input_dim = 28
input_size = input_dim**2
n_classes = 10
n_epochs = 10
learning_rate = 0.01

# Creating the model object
model = LogisticRegression(input_size, n_classes)

# Initiating loss and optimizer
# do we want SGD or Adam? read up on differences
# want something else for loss? CEL computes softmax internally

def train_epochs(model, train_dl = train_dl, n_epochs = 10, lr = 0.01, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters()) # is this necessary?
    optimizer = optim.SGD(parameters, lr = lr)
    model.train()
    for epoch in np.arange(n_epochs):
        for i, (x, y) in enumerate(train_dl):
            x = x.reshape(-1, input_size)
            out = model(x)
            loss = F.cross_entropy(out, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 2 == 0:
            print('Epoch ' + str(epoch) + ': ')
    test_loss(model, test_dl)


def test_loss(model, test_dl = test_dl):
    model.eval()
    total = 0
    correct = 0
    sum_loss = 0
    preds_list = list()

    for i, (x, y) in enumerate(test_dl):
        x = x.reshape(-1, input_size)
        batch = y.shape[0]
        out = model(x)
        loss = F.cross_entropy(out, y)
        preds = torch.argmax(out, dim = 1)
        sum_loss += batch*(loss.item())
        correct += preds.eq(y.data).sum().item()
        total += batch
        preds_list.append(preds)
        
    print("val loss and accuracy", sum_loss/total, correct/total)
    return preds_list


In [220]:
train_epochs(model, n_epochs = 6)

Epoch 0: 
Epoch 2: 
Epoch 4: 
val loss and accuracy 0.2786918731212616 0.9195


In [218]:
train_epochs(model, n_epochs = 6)

Epoch 0: 
Epoch 2: 
Epoch 4: 
val loss and accuracy 0.28449721088409424 0.9193


In [44]:
model.parameters()

<generator object Module.parameters at 0x12a81edb0>

In [159]:
x, y = next(iter(train_dl))

In [160]:
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [169]:
x = x.reshape(-1, 28**2)

In [191]:
y

tensor([ 1,  1,  0,  4,  4,  2,  2,  7,  9,  9,  1,  6,  3,  2,
         8,  9,  5,  4,  2,  5,  7,  8,  9,  1,  6,  1,  8,  5,
         1,  1,  6,  9,  2,  8,  6,  8,  0,  7,  6,  1,  9,  4,
         8,  5,  9,  6,  3,  3,  4,  3,  3,  3,  4,  8,  9,  8,
         5,  6,  6,  9,  5,  1,  7,  1])

In [171]:
out = model(x)

In [172]:
F.cross_entropy(out, y)

tensor(0.3917)

In [182]:
out

tensor([[-1.2301e+01, -4.2802e-02, -5.3405e+00, -5.1222e+00, -1.1703e+01,
         -7.4697e+00, -7.3047e+00, -7.5752e+00, -3.5515e+00, -7.2750e+00],
        [-1.0181e+01, -4.9038e-02, -6.3201e+00, -4.4479e+00, -7.4766e+00,
         -4.0729e+00, -5.7585e+00, -6.5310e+00, -4.5835e+00, -6.2715e+00],
        [-1.3986e-03, -2.1634e+01, -1.1154e+01, -9.0436e+00, -1.3884e+01,
         -7.1985e+00, -1.1881e+01, -9.6374e+00, -8.5020e+00, -8.3293e+00],
        [-6.3397e+00, -1.2844e+01, -8.9208e+00, -1.0168e+01, -5.3882e-02,
         -6.5803e+00, -5.2060e+00, -5.9307e+00, -5.7475e+00, -3.2755e+00],
        [-1.1553e+01, -1.3312e+01, -1.0907e+01, -9.8390e+00, -3.0842e-01,
         -4.7261e+00, -6.4389e+00, -9.0482e+00, -3.3685e+00, -1.5128e+00],
        [-1.5115e+01, -5.4320e+00, -4.4530e-02, -3.4295e+00, -2.0189e+01,
         -1.2131e+01, -5.1969e+00, -1.8338e+01, -6.6954e+00, -1.5369e+01],
        [-1.1881e+01, -9.6766e+00, -1.7927e-02, -4.4257e+00, -8.6833e+00,
         -1.0827e+01, -5.4028e+0

In [124]:
out.shape, y.shape

(torch.Size([64, 10]), torch.Size([64]))

In [202]:
foo = [torch.argmax(x) for x in out.data]

In [209]:
y.eq(foo)

TypeError: eq() received an invalid combination of arguments - got (list), but expected one of:
 * (Tensor other)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m)
 * (float other)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m)


In [210]:
torch.argmax(out, dim=1)

tensor([ 1,  1,  0,  4,  4,  2,  2,  7,  9,  9,  1,  6,  3,  2,
         8,  9,  4,  4,  2,  8,  7,  8,  9,  1,  6,  1,  8,  5,
         1,  0,  6,  9,  3,  1,  6,  8,  0,  7,  6,  1,  9,  4,
         8,  5,  9,  6,  3,  3,  4,  3,  3,  3,  4,  8,  3,  8,
         5,  6,  6,  9,  5,  1,  7,  1])

In [203]:
foo[0] == y[0]

tensor(1, dtype=torch.uint8)

In [204]:
foo

[tensor(1),
 tensor(1),
 tensor(0),
 tensor(4),
 tensor(4),
 tensor(2),
 tensor(2),
 tensor(7),
 tensor(9),
 tensor(9),
 tensor(1),
 tensor(6),
 tensor(3),
 tensor(2),
 tensor(8),
 tensor(9),
 tensor(4),
 tensor(4),
 tensor(2),
 tensor(8),
 tensor(7),
 tensor(8),
 tensor(9),
 tensor(1),
 tensor(6),
 tensor(1),
 tensor(8),
 tensor(5),
 tensor(1),
 tensor(0),
 tensor(6),
 tensor(9),
 tensor(3),
 tensor(1),
 tensor(6),
 tensor(8),
 tensor(0),
 tensor(7),
 tensor(6),
 tensor(1),
 tensor(9),
 tensor(4),
 tensor(8),
 tensor(5),
 tensor(9),
 tensor(6),
 tensor(3),
 tensor(3),
 tensor(4),
 tensor(3),
 tensor(3),
 tensor(3),
 tensor(4),
 tensor(8),
 tensor(3),
 tensor(8),
 tensor(5),
 tensor(6),
 tensor(6),
 tensor(9),
 tensor(5),
 tensor(1),
 tensor(7),
 tensor(1)]

In [121]:
torch.max(x)

tensor(2.8215)

In [58]:
28**2

784

In [62]:
1792/7

256.0

In [61]:
896**0.5

29.93325909419153